# CS-GY 9223-D: Deep Learning Homework 2
Due on Friday, 12th March 2019, 11:55 PM

This homework can be done in pairs.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Hupo Tang, ht1073

Member 2: Name, NetID

In [64]:
import os
import numpy as np
import glob
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD, RMSprop, Adam

# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, Lambda, Reshape, ZeroPadding2D
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.utils import to_categorical

import cv2
# import matplotlib.pylab as plt
# %matplotlib inline

In [5]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [66]:
data_path = './'
TRAIN = './images_training_rev1/'
VALID = './images_validation_rev1'
TEST = './images_test_rev1/'
LABELS = './training_solutions_rev1.csv'

## Train and validation data split

In [68]:
'''
Do not run twice!!!
'''

g = glob.glob(data_path+'images_training_rev1/*.jpg')
shuf = np.random.permutation(g)
for i in range(2000):
    os.rename(shuf[i], data_path + 'images_validation_rev1/' + shuf[i].split("/")[-1])

## Data preprocessing

In [69]:
from random import shuffle
from scipy.misc import imresize
import csv

class data_loader:    
    """
    Creates a class for handling train/valid/test data paths,
    training labels and image IDs.
    Useful for switching between sample and full datasets.
    """
    def __init__(self, path):    
        self.path = path 
        self.train_path = TRAIN
        self.val_path = VALID
        self.test_path = TEST
        
        def get_paths(directory):
            return [f for f in os.listdir(directory)]
        
        self.training_images_paths = get_paths(self.train_path)
        self.validation_images_paths = get_paths(self.val_path)
        self.test_images_paths = get_paths(self.test_path)    
        
        def get_all_solutions():
        # Import solutions file and load into self.solutions
            all_solutions = {}
            # /'training_solutions_rev1.csv'
            with open(LABELS, 'r') as f:
                reader = csv.reader(f, delimiter=",")
                next(reader)
                for i, line in enumerate(reader):
                    all_solutions[line[0]] = [float(x) for x in line[1:]]
            return all_solutions
        
        self.all_solutions = get_all_solutions()

    def get_id(self,fname):
        return fname.replace(".jpg","").replace("data","")
        
    def find_label(self,val):
        return self.all_solutions[val]

In [123]:
def process_images(paths):
    """
    Import image at 'paths', decode, centre crop and prepare for batching. 
    This is images processing for VGG Model.
    """
    count = len(paths)
    arr = np.zeros(shape=(count,3,106,106))
    for c, path in enumerate(paths):
        img = cv2.imread(path).T
        img = img[:,106:106*3,106:106*3] #crop 424x424 -> 212x212
        img = imresize(img,size=(106,106,3),interp="cubic").T # downsample to half res
        arr[c] = img
    return arr

def BatchGenerator(getter):
    while 1:
        for f in getter.training_images_paths:
            X_train = process_images([getter.train_path + '/' + fname for fname in [f]])
            X_train = np.reshape(X_train, (1,106,106,3))
            id_ = getter.get_id(f)
            y_train = np.array(getter.find_label(id_))
            y_train = np.reshape(y_train,(1,37))
            yield (X_train, y_train)

def validGenerator(getter):
    while 1:
        for f in getter.validation_images_paths:
            X_val = process_images([getter.val_path + '/' + fname for fname in [f]])
            X_val = np.reshape(X_val, (1,106,106,3))
            id_ = getter.get_id(f)
            y_val = np.array(getter.find_label(id_))
            y_val = np.reshape(y_val, (1,37))
            yield (X_val, y_val)

In [ ]:
def process_images2(paths,aug=0):
    """
    Import image at 'paths', decode, centre crop and prepare for batching. 
    """
    count = len(paths)
    arr = np.zeros(shape=(count,3,69,69))
    for c, path in enumerate(paths):
        raw_img = cv2.imread(path).T
        if not aug:
            img=raw_img
        else:
            auger={1:rotate90,2:rotate180,3:flip}.get(aug)
            img=auger(raw_img)
        img = img[:,108:315,108:315] #crop 424x424 -> 207x207
        img = imresize(img,size=(69,69,3),interp="cubic").T # downsample to half res
        arr[c] = img
    return arr

def BatchGenerator2(getter):
    while 1:
        for f in getter.training_images_paths:
            for aug in {0,1,2,3,4,5}:
                X_train = process_images([getter.train_path + '/' + fname for fname in [f]], aug)
                X_train = np.reshape(X_train,(1,69,69,3))
                id_ = getter.get_id(f)
                y_train = np.array(getter.find_label(id_))
                y_train = np.reshape(y_train,(1,37))
                assert(X_train.shape==(1,69,69,3))
                yield (X_train, y_train)

def validGenerator2(getter):
    while 1:
        for f in getter.validation_images_paths:
            X_train = process_images([getter.train_path + '/' + fname for fname in [f]], aug)
            X_train = np.reshape(X_train,(1,69,69,3))
            id_ = getter.get_id(f)
            y_train = np.array(getter.find_label(id_))
            y_train = np.reshape(y_train,(1,37))
            assert(X_train.shape==(1,69,69,3))
            yield (X_train, y_train)


### Data augmentation

In [118]:
# Expanded the original data by five times by rotating and flipping。

def rotate90(img):
    return np.rot90(img,axes=(1,2))

def rotate180(img):
    tmp = np.rot90(img,axes=(1,2))
    return np.rot90(tmp,axes=(1,2))

def rotate270(img):
    tmp = np.rot90(img,axes=(1,2))
    tmp = np.rot90(tmp,axes=(1,2))
    return np.rot90(tmp,axes=(1,2))

def flip_h(img):
    return np.flip(img, axis=1)

def flip_v(img):
    return np.flip(img, axis=2)

def process_withaug(paths,aug=0):
    count = len(paths)
    arr = np.zeros(shape=(count,3,106,106))
    for c, path in enumerate(paths):
        raw_img = cv2.imread(path).T
        if not aug:
            img=raw_img
        else:
            auger={1:rotate90,2:rotate180,3:rotate270,4:flip_h,5:flip_v}.get(aug)
            img=auger(raw_img)
        img = img[:,106:106*3,106:106*3]
        img = imresize(img,size=(106,106,3),interp="cubic").T
        arr[c] = img
    return arr

def BatchGenerator_withaug(getter):
    while 1:
        for f in getter.training_images_paths:
            for aug in {0,1,2,3,4,5}:
                X_train = process_withaug([getter.train_path + '/' + fname for fname in [f]], aug)
                X_train = np.reshape(X_train, (1,106,106,3))
                id_ = getter.get_id(f)
                y_train = np.array(getter.find_label(id_))
                y_train = np.reshape(y_train,(1,37))
                yield (X_train, y_train)


In [88]:
fetcher = data_loader(data_path)

## Build Model

### VGG

In [135]:
def ConvBlock(layers, model, filters):
    """
    Create a layered Conv/Pooling block
    """
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))  # zero padding of size 1
        model.add(Convolution2D(filters, 3, 3, activation='relu'))  # 3x3 filter size 
    model.add(MaxPooling2D((1,1), strides=(2,2)))

def FCBlock(model):
    """
    Fully connected block with ReLU and dropout
    """
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG_16():
    """
    Implement VGG16 architecture
    """
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(106,106,3)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    
    model.add(Dense(37, activation = 'sigmoid'))
    return model


In [119]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

In [137]:
# Compile 
optimizer = RMSprop(lr=1e-6)
model = VGG_16()
model.compile(loss='mean_squared_error', optimizer=optimizer)

### My CNN design 

In [ ]:
def MyCNN():
    model = Sequential()
    #NHWC(,69,69,3)
    model.add(Lambda(lambda x: x, input_shape=(69,69,3)))
    print(model.output_shape)    
#     model.add(ZeroPadding2D((2,2)))
    model.add(Convolution2D(32, 6, 6, activation='relu'))

    model.add(MaxPooling2D((2,2), strides=(2,2)))
#     model.add(ZeroPadding2D((2,2)))
    model.add(Convolution2D(64, 5, 5, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(37, activation='softmax'))
    return model

In [127]:
early_stopping = EarlyStopping(monitor='val_loss', patience=12, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath='./tmp/weights.hdf5', verbose=1, save_best_only=True)

batch_size = 32
steps_to_take = int(len(fetcher.training_images_paths)/batch_size)
val_steps_to_take = int(len(fetcher.validation_images_paths)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

history = LossHistory()

In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

## Train Model

In [ ]:
# VGG16 with no augmentation

hist = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=50,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )
# 0.021

In [ ]:
# VGG16 with augmentation

hist_aug = model.fit_generator(BatchGenerator_withaug(fetcher),
                    samples_per_epoch=steps_to_take*3, 
                    nb_epoch=25,
                    verbose=2,
                    callbacks=[hist,checkpointer,early_stopping],
                   )
# 0.017

In [31]:
#Aug with larger epochs

hist_aug2 = model.fit_generator(BatchGenerator_withaug(fetcher),
                    samples_per_epoch=steps_to_take*3, 
                    nb_epoch=50,
                    verbose=2,
                    callbacks=[hist,checkpointer,early_stopping],
                   )
# 0.015

Epoch 1/50
 - 116s - loss: 0.0173
Epoch 2/50
 - 116s - loss: 0.0170
Epoch 3/50
 - 116s - loss: 0.0163
Epoch 4/50
 - 116s - loss: 0.0166
Epoch 5/50
 - 116s - loss: 0.0168
Epoch 6/50
 - 116s - loss: 0.0169
Epoch 7/50
 - 116s - loss: 0.0168
Epoch 8/50
 - 117s - loss: 0.0165
Epoch 9/50
 - 116s - loss: 0.0164
Epoch 10/50
 - 116s - loss: 0.0167
Epoch 11/50
 - 116s - loss: 0.0163
Epoch 12/50
 - 116s - loss: 0.0161
Epoch 13/50
 - 116s - loss: 0.0164
Epoch 14/50
 - 116s - loss: 0.0164
Epoch 15/50
 - 116s - loss: 0.0165
Epoch 16/50
 - 116s - loss: 0.0160
Epoch 17/50
 - 116s - loss: 0.0155
Epoch 18/50
 - 116s - loss: 0.0164
Epoch 19/50
 - 116s - loss: 0.0156
Epoch 20/50
 - 116s - loss: 0.0158
Epoch 21/50
 - 116s - loss: 0.0155
Epoch 22/50
 - 116s - loss: 0.0162
Epoch 23/50
 - 116s - loss: 0.0161
Epoch 24/50
 - 116s - loss: 0.0156
Epoch 25/50
 - 116s - loss: 0.0158
Epoch 26/50
 - 118s - loss: 0.0153
Epoch 27/50
 - 122s - loss: 0.0158
Epoch 28/50
 - 133s - loss: 0.0155
Epoch 29/50
 - 123s - loss: 0

In [138]:
hist_v = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take,
                    nb_epoch=50,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
#                     validation_data=validGenerator(fetcher),
#                     validation_steps=val_steps_to_take
                   )

Epoch 1/50
 - 56s - loss: 0.0538
Epoch 2/50
 - 54s - loss: 0.0327
Epoch 3/50
 - 54s - loss: 0.0310
Epoch 4/50
 - 54s - loss: 0.0298
Epoch 5/50
 - 54s - loss: 0.0285
Epoch 6/50
 - 54s - loss: 0.0270
Epoch 7/50
 - 54s - loss: 0.0267
Epoch 8/50
 - 54s - loss: 0.0257
Epoch 9/50
 - 54s - loss: 0.0248
Epoch 10/50
 - 54s - loss: 0.0248
Epoch 11/50
 - 54s - loss: 0.0237
Epoch 12/50
 - 54s - loss: 0.0231
Epoch 13/50
 - 54s - loss: 0.0228
Epoch 14/50
 - 54s - loss: 0.0224
Epoch 15/50
 - 54s - loss: 0.0221
Epoch 16/50
 - 54s - loss: 0.0220
Epoch 17/50
 - 54s - loss: 0.0214
Epoch 18/50
 - 54s - loss: 0.0221
Epoch 19/50
 - 54s - loss: 0.0209
Epoch 20/50
 - 54s - loss: 0.0210
Epoch 21/50
 - 54s - loss: 0.0209
Epoch 22/50
 - 54s - loss: 0.0207
Epoch 23/50
 - 54s - loss: 0.0211
Epoch 24/50
 - 54s - loss: 0.0207
Epoch 25/50
 - 54s - loss: 0.0204
Epoch 26/50
 - 54s - loss: 0.0198
Epoch 27/50
 - 54s - loss: 0.0199
Epoch 28/50
 - 55s - loss: 0.0198
Epoch 29/50
 - 57s - loss: 0.0204
Epoch 30/50
 - 55s - lo

In [141]:
hist_vaug = model.fit_generator(BatchGenerator_withaug(fetcher),
                    samples_per_epoch=steps_to_take,
                    nb_epoch=50,
                    verbose=2,
                    callbacks=[hist_v2,checkpointer,early_stopping],
                    validation_data=validGenerator(fetcher),
                    validation_steps=val_steps_to_take
                   )

Epoch 1/50
 - 54s - loss: 0.0161 - val_loss: 0.0176

Epoch 00001: val_loss did not improve from 0.01435
Epoch 2/50
 - 54s - loss: 0.0159 - val_loss: 0.0140

Epoch 00002: val_loss improved from 0.01435 to 0.01405, saving model to ./tmp/weights.hdf5
Epoch 3/50
 - 54s - loss: 0.0154 - val_loss: 0.0155

Epoch 00003: val_loss did not improve from 0.01405
Epoch 4/50
 - 54s - loss: 0.0154 - val_loss: 0.0140

Epoch 00004: val_loss improved from 0.01405 to 0.01400, saving model to ./tmp/weights.hdf5
Epoch 5/50
 - 54s - loss: 0.0155 - val_loss: 0.0156

Epoch 00005: val_loss did not improve from 0.01400
Epoch 6/50
 - 54s - loss: 0.0158 - val_loss: 0.0159

Epoch 00006: val_loss did not improve from 0.01400
Epoch 7/50
 - 54s - loss: 0.0160 - val_loss: 0.0145

Epoch 00007: val_loss did not improve from 0.01400
Epoch 8/50
 - 54s - loss: 0.0157 - val_loss: 0.0149

Epoch 00008: val_loss did not improve from 0.01400
Epoch 9/50
 - 54s - loss: 0.0155 - val_loss: 0.0164

Epoch 00009: val_loss did not impro

In [151]:
from keras.models import load_model
Model = load_model('tmp/weights.hdf5')

In [ ]:
hist_vaug = model.fit_generator(BatchGenerator_withaug(fetcher),
                    samples_per_epoch=steps_to_take*6,
                    nb_epoch=30,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                    validation_data=validGenerator(fetcher),
                    validation_steps=val_steps_to_take
                   )

Epoch 1/30
 - 327s - loss: 0.0134 - val_loss: 0.0245

Epoch 00001: val_loss did not improve from 0.01177
Epoch 2/30
 - 327s - loss: 0.0130 - val_loss: 0.0186

Epoch 00002: val_loss did not improve from 0.01177
Epoch 3/30
 - 326s - loss: 0.0123 - val_loss: 0.0176

Epoch 00003: val_loss did not improve from 0.01177
Epoch 4/30
 - 327s - loss: 0.0124 - val_loss: 0.0181

Epoch 00004: val_loss did not improve from 0.01177
Epoch 5/30
 - 326s - loss: 0.0123 - val_loss: 0.0178

Epoch 00005: val_loss did not improve from 0.01177
Epoch 6/30
 - 326s - loss: 0.0122 - val_loss: 0.0203

Epoch 00006: val_loss did not improve from 0.01177
Epoch 7/30
 - 326s - loss: 0.0125 - val_loss: 0.0163

Epoch 00007: val_loss did not improve from 0.01177
Epoch 8/30
 - 327s - loss: 0.0120 - val_loss: 0.0172

Epoch 00008: val_loss did not improve from 0.01177
Epoch 9/30
 - 326s - loss: 0.0117 - val_loss: 0.0237

Epoch 00009: val_loss did not improve from 0.01177
Epoch 10/30
 - 327s - loss: 0.0119 - val_loss: 0.0192



In [147]:
def testGenerator(getter):
    while 1:
        for f in getter.test_images_paths:
            X_test = process_images([getter.test_path + '/' + fname for fname in [f]])
            X_test = np.reshape(X_test, (1,106,106,3))
            yield (X_test)

In [148]:
predictions = model.predict_generator(testGenerator(fetcher),
                       val_samples = len(fetcher.test_images_paths),
                        max_q_size = 32)

In [149]:
predictions.shape

(600, 37)

In [ ]:
header = open('all_zeros_benchmark.csv','r').readlines()[0]

with open('submission_3.csv','w') as outfile:
    outfile.write(header)
    for i in range(len(fetcher.test_images_paths)):
        id_ = (fetcher.get_id(fetcher.test_images_paths[i]))
        pred = predictions[i]
        outline = id_ + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")